In [29]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

In [30]:
# Recordar obtener los URL "RAW" en Github

dataSet_url = "https://raw.githubusercontent.com/francopiria/Beer-Data-Science/main/beer_profile_and_ratings.csv"

In [ ]:
df = pd.read_csv(dataSet_url, index_col=3)
print(df.columns)
print(len(df))
# print(df.head(3))

# Largo: 3197
# Columnas
#       ['Name', 'Style', 'Brewery', 'Description',
#       'ABV', 'Min IBU', 'Max IBU',
#       'Astringency', 'Body', 'Alcohol', 'Bitter', 'Sweet', 'Sour', 'Salty',
#       'Fruits', 'Hoppy', 'Spices', 'Malty',
#       'review_aroma','review_appearance', 'review_palate', 'review_taste', 'review_overall',
#       'number_of_reviews']

# ABV es Alcohol por volumen, IBU Amargor, "Alcohol" es un descriptor

#Ideas
# Analizar ABV contra descriptor alcohol
# IBU contra astringencia, cuerpo, bitter y sour
# Cada review específico contra el overall para ver cuales afecntan más

In [ ]:
# Normalizar las varialbes
# 'Astringency', 'Body', 'Alcohol', 'Bitter', 'Sweet',
# 'Sour','Salty','Fruits', 'Hoppy', 'Spices', 'Malty'
# A una escala del 1 al 100 cada una

# Lista de columnas a redimensionar
columnas_redimensionar = ['Astringency', 'Body', 'Alcohol', 'Bitter', 'Sweet', 'Sour', 'Salty', 'Fruits', 'Hoppy', 'Spices', 'Malty']

# Máximos asociados a cada campo según el Dataset
maximos = {
    'Astringency': 89,
    'Body': 97,
    'Alcohol': 64,
    'Bitter': 86,
    'Sweet': 128,
    'Sour': 111,
    'Salty': 13,
    'Fruits': 174,
    'Hoppy': 148,
    'Spices': 99,
    'Malty': 209
}

# Función para redimensionar los valores a la escala de 0 a 100
def redimensionar(valor, max_valor):
    nuevo_valor = (valor / max_valor) * 100
    return nuevo_valor


# Aplicamos la función de redimensionamiento a las columnas usando un bucle for
for columna in columnas_redimensionar:
    df[columna] = redimensionar(df[columna], maximos[columna])

# Visualizamos el DataFrame con las columnas redimensionadas
# print(df.head(3))

In [ ]:
# Ahora quiero crear una nueva colummna que sea el promedio entre el mínimo y máximo IBU para trabajar con este valor


In [ ]:
#Ahora el DataSet tiene dimensiones adecuadas para sus diferentes campos.
#

# Calcular el número de reviews que corresponde al 5% más bajo
percentile_10 = int(0.10 * len(df))  # Obtener el 5% de las entradas

# Ordenar el DataFrame por el número de reviews y seleccionar el 5% más bajo
lowest_reviews = df.sort_values('number_of_reviews').head(percentile_10)

print(percentile_10)
print(lowest_reviews)

In [ ]:
#observo que los datos pertenecientes al 10%  con menos numero de reviews tienen como máximo 5 reviews.
# Decido cortar del dataset todas las entradas que tengan 5 reviews o menos.

# Filtrar el DataFrame para quedarse con las entradas que tienen 6 o más reviews
df_filtered = df[df['number_of_reviews'] >= 5]

# Ver las primeras filas del DataFrame filtrado
print(df_filtered)

In [ ]:
score_variables_names = [
    "review_aroma",
    "review_appearance",
    "review_palate",
    "review_taste",
    "review_overall",
    "number_of_reviews"
]

In [ ]:
# Histogramas para evaluar la distribución de las variables de reviews

for columna in score_variables_names:
    plt.figure(figsize=(6, 4))  # Tamaño de la figura
    plt.hist(df[columna])
    plt.xlabel(f'Valores {columna}')
    plt.ylabel('Frecuencia')
    plt.title(f'Histograma de {columna}')
    plt.show()

In [ ]:
# Gráfico de barras de estilos de cerveza para evaluar cuales son las cervezas más producidas
plt.figure(figsize=(10, 6))
style_counts = df['Style'].value_counts().head(10)  # Top 10 estilos más comunes
sns.barplot(x=style_counts.values, y=style_counts.index, palette='viridis')
plt.xlabel('Cantidad')
plt.ylabel('Estilo de Cerveza')
plt.title('Cantidad de Cervezas por Estilo (Top 10)')
plt.show()

In [ ]:
# Diagrama de dispersión ABV vs. calificaciones para evaluar correlación alcohol calificación
plt.figure(figsize=(8, 6))
sns.scatterplot(x='ABV', y='review_overall', data=df)
plt.xlabel('ABV')
plt.ylabel('Rating')
plt.title('Relación entre ABV y Calificaciones')
plt.show()

# No parece haber correlación clara

In [ ]:
# Diagrama de dispersión IBU vs. calificaciones
plt.figure(figsize=(8, 6))
sns.scatterplot(x='Min IBU', y='review_overall', data=df)
plt.xlabel('Min IBU')
plt.ylabel('Rating')
plt.title('Relación entre Min IBU y Calificaciones')
plt.show()

plt.figure(figsize=(8, 6))
sns.scatterplot(x='Max IBU', y='review_overall', data=df)
plt.xlabel('Max IBU')
plt.ylabel('Rating')
plt.title('Relación entre Max IBU y Calificaciones')
plt.show()

# Se ve que no hay cervezas de nota baja con alto amargor.
# Dentro de las de notas mas altas, hay de bajo y alto amargor
# Pendiente hacer el gráfico con el promedio de mínimo y máximo amargor

In [ ]:
# Mapa de calor para correlaciones
plt.figure(figsize=(8, 6))
correlation_matrix = df[['ABV', 'Min IBU', 'review_overall']].corr()
sns.heatmap(correlation_matrix, annot=True, cmap='coolwarm', fmt=".2f")
plt.title('Mapa de Calor de Correlaciones')
plt.show()

plt.figure(figsize=(8, 6))
correlation_matrix = df[['ABV', 'Max IBU', 'review_overall']].corr()
sns.heatmap(correlation_matrix, annot=True, cmap='coolwarm', fmt=".2f")
plt.title('Mapa de Calor de Correlaciones')
plt.show()

# Sería bueno ver correlaciones entre review overall con review aroma/amargor/etc, pendiente

In [ ]:
# Gráfico de densidad de calificaciones por estilo de cerveza

plt.figure(figsize=(10, 6))
for style in df['Style'].unique():
    sns.kdeplot(data=df[df['Style'] == style]['review_overall'], label=style, fill=True, common_norm=False)
    
plt.xlabel('Calificación')
plt.ylabel('Densidad')
plt.title('Distribución de Calificaciones por Estilo de Cerveza')
plt.legend(title='Estilo de Cerveza', bbox_to_anchor=(1.05, 1), loc='upper left')
plt.show()

# Dificil de ver por ser tantos estilos
# Filtro el dataset para ver las mejores y las peores

In [ ]:
# Calcular la media de calificaciones por estilo de cerveza
mean_ratings = df.groupby('Style')['review_overall'].mean().sort_values(ascending=False)

# Obtener los 10 estilos con las calificaciones más altas y más bajas
top_styles = mean_ratings.head(10).index.tolist()
bottom_styles = mean_ratings.tail(10).index.tolist()

# Filtrar el DataFrame para obtener los datos correspondientes a los estilos seleccionados
selected_styles = top_styles + bottom_styles
filtered_df = df[df['Style'].isin(selected_styles)]

# Gráfico de densidad para los estilos seleccionados
plt.figure(figsize=(10, 6))
for style in selected_styles:
    sns.kdeplot(data=filtered_df[filtered_df['Style'] == style]['review_overall'], label=style, fill=True)

plt.xlabel('Calificación')
plt.ylabel('Densidad')
plt.title('Distribución de Calificaciones por Estilo de Cerveza')
plt.legend(title='Estilo de Cerveza', bbox_to_anchor=(1.05, 1), loc='upper left')
plt.show()

In [ ]:
# Calcular la media de calificaciones por estilo de cerveza
mean_ratings = df.groupby('Style')['review_overall'].mean().sort_values(ascending=False)

# Obtener los 5 estilos con las calificaciones más altas y más bajas
top_styles = mean_ratings.head(5).index.tolist()
bottom_styles = mean_ratings.tail(5).index.tolist()

# Filtrar el DataFrame para obtener los datos correspondientes a los estilos seleccionados
selected_styles = top_styles + bottom_styles
filtered_df = df[df['Style'].isin(selected_styles)]

# Gráfico de densidad para los estilos seleccionados
plt.figure(figsize=(10, 6))
for style in selected_styles:
    sns.kdeplot(data=filtered_df[filtered_df['Style'] == style]['review_overall'], label=style, fill=True)

plt.xlabel('Calificación')
plt.ylabel('Densidad')
plt.title('Distribución de Calificaciones por Estilo de Cerveza')
plt.legend(title='Estilo de Cerveza', bbox_to_anchor=(1.05, 1), loc='upper left')
plt.show()

# Las low alcohol beer parecen ser de las peor calificadas
# New england IPA parece ser de las mejor calificadas